In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [10]:
# https://python.langchain.com/docs/modules/model_io/output_parsers/retry
# https://python.langchain.com/docs/modules/model_io/output_parsers/structured

from langchain.prompts import (
    PromptTemplate,
)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
    RetryOutputParser,
)
from pydantic import BaseModel, Field, validator
from typing import List


In [11]:

template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""

class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

In [12]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [13]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

bad_response = '{"action": "search"}'
parser.parse(bad_response)



OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  field required (type=value_error.missing)

In [16]:
import sys
sys.path.append('../developer')

from _lessonshelper.pretty_print_callback_handler import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()


fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(callbacks=[callback]))
fix_parser.parse(bad_response)


[chat_model][start] - prompts : Instructions:
--------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"action": {"title": "Action", "description": "action to take", "type": "string"}, "action_input": {"title": "Action Input", "description": "input to the action", "type": "string"}}, "required": ["action", "action_input"]}
```
--------------
Completion:
--------------
{"action": "search"}
--------------

Above, the Completion did not satisfy the constraints given in the Instructions.
Error:
--------------
OutputParserException('Failed to parse Action from completion {

Action(action='search', action_input='query')

In [ ]:
from langchain.output_parsers import RetryWithErrorOutputParser

retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0, callbacks=[callback])
)

retry_parser.parse_with_prompt(bad_response, prompt_value)

